## No Communication

In [5]:
import pygame
import pymunk
import pymunk.pygame_util
import sys
import random

# --- Constants ---
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
AGENT_RADIUS = 15
MAX_SPEED = 150  # pixels/sec
GOAL_THRESHOLD = 20 # How close to a waypoint to be "at" it

# --- RRT Planner Constants ---
RRT_ITERATIONS = 500
RRT_STEP_SIZE = 25  # How far to extend the tree in each step
RRT_GOAL_BIAS = 0.1 # Probability of sampling the goal (0.0 to 1.0)

# --- Pygame Setup ---
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Two RRT Agents (No Communication)")
clock = pygame.time.Clock()

# --- Pymunk Setup ---
space = pymunk.Space()
space.gravity = (0, 0) # No gravity for a top-down view
draw_options = pymunk.pygame_util.DrawOptions(screen)

random.seed(5)

class RRTPlanner:
    """
    A simple RRT (Rapidly-exploring Random Tree) planner.
    Plans a path avoiding only a given set of static obstacles.
    """
    
    class Node:
        """Helper class for RRT, represents a point in the tree."""
        def __init__(self, pos):
            self.pos = pymunk.Vec2d(*pos)
            self.parent = None

    def __init__(self, start_pos, goal_pos, static_obstacles, bounds):
        self.start_node = self.Node(start_pos)
        self.goal_node = self.Node(goal_pos)
        self.nodes = [self.start_node]
        self.static_obstacles = static_obstacles # List of pymunk.Segment shapes
        self.bounds = bounds # (width, height)

    def plan(self):
        """Generates a path from start to goal."""
        for _ in range(RRT_ITERATIONS):
            # 1. Sample a random point
            if random.random() < RRT_GOAL_BIAS:
                rand_pos = self.goal_node.pos
            else:
                rand_pos = (random.randint(0, self.bounds[0]), 
                            random.randint(0, self.bounds[1]))

            # 2. Find the nearest node in the tree
            nearest_node = min(self.nodes, key=lambda n: (n.pos - rand_pos).length_squared)

            # 3. Steer from nearest node towards the random point
            dir_vec = (rand_pos - nearest_node.pos).normalized()
            new_pos = nearest_node.pos + dir_vec * RRT_STEP_SIZE

            # 4. Check for collisions
            if self._is_collision_free(nearest_node.pos, new_pos):
                new_node = self.Node(new_pos)
                new_node.parent = nearest_node
                self.nodes.append(new_node)

                # 5. Check if goal is reachable from the new node
                if new_node.pos.get_distance(self.goal_node.pos) < RRT_STEP_SIZE:
                    if self._is_collision_free(new_node.pos, self.goal_node.pos):
                        self.goal_node.parent = new_node
                        return self._reconstruct_path()
        
        print("RRT failed to find a path.")
        return None # Failed to find path

    def _is_collision_free(self, p1, p2):
        """Check if the line segment (p1, p2) intersects any static obstacles."""
        for obs in self.static_obstacles:
            # Check if query returns an object (hit) or None (no hit)
            if obs.segment_query(p1, p2):
                return False # Not collision-free
        return True # Is collision-free

    def _reconstruct_path(self):
        """Backtrack from goal to start to get the final path."""
        path = []
        curr = self.goal_node
        while curr:
            path.append(curr.pos)
            curr = curr.parent
        # Reverse the path to go from start -> goal
        return path[::-1]


class Agent:
    """
    Represents an agent with a Pymunk body and an RRT planner.
    """
    def __init__(self, space, start_pos, goal_pos, static_obstacles, world_bounds, color="blue"):
        # Physics Setup
        mass = 1
        moment = pymunk.moment_for_circle(mass, 0, AGENT_RADIUS)
        self.body = pymunk.Body(mass, moment)
        self.body.position = start_pos
        self.shape = pymunk.Circle(self.body, AGENT_RADIUS)
        self.shape.elasticity = 0.8
        # --- REMOVED: self.shape.collision_type = 1 ---
        self.shape.color = pygame.color.THECOLORS[color]
        space.add(self.body, self.shape)
        
        # Planning Setup
        self.goal = pymunk.Vec2d(*goal_pos)
        self.path = None  # Will hold the list of Vec2d waypoints from RRT
        self.path_index = 0
        self.static_obstacles = static_obstacles
        self.world_bounds = world_bounds
        self.planner = RRTPlanner(start_pos, goal_pos, self.static_obstacles, self.world_bounds)

    def plan_path(self):
        """Run the RRT planner to generate a path."""
        print(f"Agent ({self.shape.color}) is planning a path...")
        self.path = self.planner.plan()
        if self.path:
            self.path_index = 0
            print(f"Agent ({self.shape.color}) found a path with {len(self.path)} waypoints.")
        else:
            print(f"Agent ({self.shape.color}) FAILED to find a path.")

    def update(self):
        """
        This is the "path following" controller.
        It moves the agent towards the next waypoint in its path.
        """
        if not self.path or self.path_index >= len(self.path):
            self.body.velocity = (0, 0) # No path or path is finished
            return 

        # Get the current target waypoint
        target_pos = self.path[self.path_index]
        pos = self.body.position
        
        vec_to_target = target_pos - pos
        dist_to_target = vec_to_target.length

        # Check if we've arrived at the waypoint
        if dist_to_target < GOAL_THRESHOLD:
            # We've reached this waypoint, move to the next one
            self.path_index += 1
            if self.path_index >= len(self.path):
                print(f"Agent ({self.shape.color}) reached goal!")
                self.body.velocity = (0, 0)
        else:
            # Move towards the waypoint at max speed
            direction = vec_to_target.normalized()
            self.body.velocity = direction * MAX_SPEED

def create_boundaries(space, width, height):
    """Creates static walls around the simulation area and returns them."""
    static_body = space.static_body
    walls = [
        pymunk.Segment(static_body, (0, 0), (width, 0), 2),       # Top
        pymunk.Segment(static_body, (0, 0), (0, height), 2),      # Left
        pymunk.Segment(static_body, (width, 0), (width, height), 2), # Right
        pymunk.Segment(static_body, (0, height), (width, height), 2) # Bottom
    ]
    for w in walls:
        w.elasticity = 0.5
    space.add(*walls)
    return walls # Return the walls so the planner can avoid them

def main():
    """Main simulation loop for Demo 1"""
    
    # --- Environment Setup ---
    # Create walls and get them for the planner
    static_obstacles = create_boundaries(space, SCREEN_WIDTH, SCREEN_HEIGHT)
    
    # --- Agent Setup ---
    # Swapped start/goal positions to force a collision course
    start_1 = (100, 100)
    goal_1 = (700, 500)
    
    start_2 = (700, 500)
    goal_2 = (100, 100)

    agent1 = Agent(space, start_1, goal_1, static_obstacles, (SCREEN_WIDTH, SCREEN_HEIGHT), "blue")
    agent2 = Agent(space, start_2, goal_2, static_obstacles, (SCREEN_WIDTH, SCREEN_HEIGHT), "red")
    agents = [agent1, agent2]

    # --- Initial Planning (Non-Communicative) ---
    # Each agent plans its path *without* knowing about the other agent.
    agent1.plan_path()
    agent2.plan_path() 

    # --- Simulation Loop ---
    running = True
    while running:
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Update Agents (This just follows the pre-planned path)
        for agent in agents:
            agent.update()

        # Pymunk Physics Step
        dt = 1.0 / 60.0
        space.step(dt)

        # Drawing
        screen.fill((255, 255, 255))  # White background
        space.debug_draw(draw_options) # Draw physics objects (agents, walls)
        
        # Draw paths and goals
        for agent in agents:
            # Draw the goal (green circle)
            pygame.draw.circle(screen, (0, 255, 0), (int(agent.goal.x), int(agent.goal.y)), 5)
            
            # Draw the planned path
            if agent.path:
                path_points = [(int(p.x), int(p.y)) for p in agent.path]
                pygame.draw.lines(screen, agent.shape.color, False, path_points, 2)

        pygame.display.flip()
        clock.tick(60)
        
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

Agent ((0, 0, 255, 255)) is planning a path...
Agent ((0, 0, 255, 255)) found a path with 38 waypoints.
Agent ((255, 0, 0, 255)) is planning a path...
Agent ((255, 0, 0, 255)) found a path with 37 waypoints.


KeyboardInterrupt: 

## With Communication

In [ ]:
import pygame
import pymunk
import pymunk.pygame_util
import sys
import math
import random

# --- Constants ---
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
AGENT_RADIUS = 15
MAX_SPEED = 150  # pixels/sec
GOAL_THRESHOLD = 20 # How close to a waypoint to be "at" it

# --- RRT Planner Constants ---
RRT_ITERATIONS = 500
RRT_STEP_SIZE = 25  # How far to extend the tree in each step
RRT_GOAL_BIAS = 0.1 # Probability of sampling the goal (0.0 to 1.0)
AVOIDANCE_RADIUS = AGENT_RADIUS * 2.5 # Safety buffer for dynamic obstacles

# --- Predictive Collision Checking Constants ---
PREDICTION_HORIZON = 5.0 # How many seconds into the future to check
PREDICTION_DT = 0.1      # Time step for the prediction
COLLISION_DISTANCE = AGENT_RADIUS * 2.2 # Safety buffer for predicted collision

# --- HYBRID TRIGGER ---
COMMUNICATION_TRIGGER_DISTANCE = 250 # "Warning zone" to start checking
REPLAN_COOLDOWN_TIME = 1.5           # Seconds between re-plans

# --- Pygame Setup ---
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Demo 8: Re-planning with Remaining Path")
clock = pygame.time.Clock()

# --- Pymunk Setup ---
space = pymunk.Space()
space.gravity = (0, 0) # No gravity for a top-down view
draw_options = pymunk.pygame_util.DrawOptions(screen)

random.seed(5)

class RRTPlanner:
    """
    Modified RRT planner that can avoid both static walls
    and dynamic paths from other agents.
    (This class is unchanged)
    """
    class Node:
        """Helper class for RRT, represents a point in the tree."""
        def __init__(self, pos):
            self.pos = pymunk.Vec2d(*pos)
            self.parent = None

    def __init__(self, start_pos, goal_pos, static_obstacles, bounds):
        self.start_node = self.Node(start_pos)
        self.goal_node = self.Node(goal_pos)
        self.nodes = [self.start_node]
        self.static_obstacles = static_obstacles # List of pymunk.Segment shapes
        self.bounds = bounds # (width, height)
        self.dynamic_obstacles = [] # Will be a list of paths [[Vec2d, ...], ...]

    def set_dynamic_obstacles(self, paths):
        """Sets the list of paths to avoid."""
        self.dynamic_obstacles = paths

    def plan(self):
        """Generates a path from start to goal."""
        for _ in range(RRT_ITERATIONS):
            # 1. Sample a random point
            if random.random() < RRT_GOAL_BIAS:
                rand_pos = self.goal_node.pos
            else:
                rand_pos = (random.randint(0, self.bounds[0]), 
                            random.randint(0, self.bounds[1]))

            # 2. Find the nearest node in the tree
            nearest_node = min(self.nodes, key=lambda n: (n.pos - rand_pos).length_squared)

            # 3. Steer from nearest node towards the random point
            dir_vec = (rand_pos - nearest_node.pos).normalized()
            new_pos = nearest_node.pos + dir_vec * RRT_STEP_SIZE

            # 4. Check for collisions (now checks dynamic obstacles too)
            if self._is_collision_free(nearest_node.pos, new_pos):
                new_node = self.Node(new_pos)
                new_node.parent = nearest_node
                self.nodes.append(new_node)

                # 5. Check if goal is reachable from the new node
                if new_node.pos.get_distance(self.goal_node.pos) < RRT_STEP_SIZE:
                    if self._is_collision_free(new_node.pos, self.goal_node.pos):
                        self.goal_node.parent = new_node
                        return self._reconstruct_path()
        
        return None # Failed to find path

    def _is_collision_free(self, p1, p2):
        """Check if the line segment (p1, p2) intersects any obstacles."""
        
        # 1. Check static walls
        for obs in self.static_obstacles:
            if obs.segment_query(p1, p2):
                return False # Not collision-free

        # 2. Check dynamic paths
        for path in self.dynamic_obstacles:
            for point in path:
                line_vec = p2 - p1
                if line_vec.length_squared == 0: continue 
                
                point_vec = point - p1
                t = line_vec.dot(point_vec) / line_vec.length_squared
                t = max(0, min(1, t)) # Clamp to the segment [0, 1]
                
                closest_point_on_segment = p1 + t * line_vec
                
                if closest_point_on_segment.get_distance(point) < AVOIDANCE_RADIUS:
                    return False # Not collision-free
                    
        return True # Is collision-free

    def _reconstruct_path(self):
        """Backtrack from goal to start to get the final path."""
        path = []
        curr = self.goal_node
        while curr:
            path.append(curr.pos)
            curr = curr.parent
        return path[::-1]


class Agent:
    """
    Represents an agent with a Pymunk body and an RRT planner.
    (This class is unchanged)
    """
    def __init__(self, space, start_pos, goal_pos, static_obstacles, world_bounds, color="blue"):
        # Physics Setup
        mass = 1
        moment = pymunk.moment_for_circle(mass, 0, AGENT_RADIUS)
        self.body = pymunk.Body(mass, moment)
        self.body.position = start_pos
        self.shape = pymunk.Circle(self.body, AGENT_RADIUS)
        self.shape.elasticity = 0.8
        self.shape.color = pygame.color.THECOLORS[color]
        space.add(self.body, self.shape)
        
        # Planning Setup
        self.goal = pymunk.Vec2d(*goal_pos)
        self.path = None  
        self.path_index = 0
        self.static_obstacles = static_obstacles
        self.world_bounds = world_bounds
        self.planner = RRTPlanner(start_pos, goal_pos, self.static_obstacles, self.world_bounds)

    def plan_path_non_destructive(self):
        """
        Runs the RRT planner. If it finds a new path, it adopts it.
        If it fails, it KEEPS its old path and does not stop.
        """
        new_path = self.planner.plan()
        
        if new_path:
            # Success! Adopt the new path.
            self.path = new_path
            self.path_index = 0 # Reset path following
            print(f"Agent ({self.shape.color}) found a NEW path with {len(self.path)} waypoints.")
        else:
            # Failure. Do nothing. Keep the old path.
            print(f"Agent ({self.shape.color}) re-plan FAILED. Sticking to old path.")
            pass 

    def plan_path_initial(self):
        """Run the RRT planner for the first time."""
        print(f"Agent ({self.shape.color}) is planning an initial path...")
        self.path = self.planner.plan()
        if self.path:
            self.path_index = 0
            print(f"Agent ({self.shape.color}) found an initial path with {len(self.path)} waypoints.")
        else:
            print(f"Agent ({self.shape.color}) FAILED to find an initial path.")

    def update(self):
        """This is the "path following" controller."""
        if not self.path or self.path_index >= len(self.path):
            self.body.velocity = (0, 0) # No path or path is finished
            return 

        target_pos = self.path[self.path_index]
        pos = self.body.position
        vec_to_target = target_pos - pos
        dist_to_target = vec_to_target.length

        if dist_to_target < GOAL_THRESHOLD:
            self.path_index += 1
            if self.path_index >= len(self.path):
                print(f"Agent ({self.shape.color}) reached goal!")
                self.body.velocity = (0, 0)
        else:
            direction = vec_to_target.normalized()
            self.body.velocity = direction * MAX_SPEED

def create_boundaries(space, width, height):
    """Creates static walls around the simulation area and returns them."""
    static_body = space.static_body
    walls = [
        pymunk.Segment(static_body, (0, 0), (width, 0), 2),       # Top
        pymunk.Segment(static_body, (0, 0), (0, height), 2),      # Left
        pymunk.Segment(static_body, (width, 0), (width, height), 2), # Right
        pymunk.Segment(static_body, (0, height), (width, height), 2) # Bottom
    ]
    for w in walls:
        w.elasticity = 0.5
    space.add(*walls)
    return walls 

def predict_collision(agent1, agent2, horizon_seconds):
    """
    Simulates agents' movement along their paths to check for future collisions.
    (This function is unchanged)
    """
    if not agent1.path or agent1.path_index >= len(agent1.path) or \
       not agent2.path or agent2.path_index >= len(agent2.path):
        return False

    sim_pos1 = pymunk.Vec2d(*agent1.body.position)
    sim_path_index1 = agent1.path_index
    sim_pos2 = pymunk.Vec2d(*agent2.body.position)
    sim_path_index2 = agent2.path_index

    current_time = 0.0
    while current_time < horizon_seconds:
        # --- Simulate Agent 1 ---
        if sim_path_index1 < len(agent1.path):
            target1 = agent1.path[sim_path_index1]
            vec_to_target1 = target1 - sim_pos1
            dist_to_target1 = vec_to_target1.length
            move_dist = MAX_SPEED * PREDICTION_DT
            if move_dist >= dist_to_target1:
                sim_pos1 = target1
                sim_path_index1 += 1
            else:
                sim_pos1 += vec_to_target1.normalized() * move_dist
        
        # --- Simulate Agent 2 ---
        if sim_path_index2 < len(agent2.path):
            target2 = agent2.path[sim_path_index2]
            vec_to_target2 = target2 - sim_pos2
            dist_to_target2 = vec_to_target2.length
            move_dist = MAX_SPEED * PREDICTION_DT
            if move_dist >= dist_to_target2:
                sim_pos2 = target2
                sim_path_index2 += 1
            else:
                sim_pos2 += vec_to_target2.normalized() * move_dist

        # --- Check for Collision ---
        if sim_pos1.get_distance(sim_pos2) < COLLISION_DISTANCE:
            return True
            
        if sim_path_index1 >= len(agent1.path) and sim_path_index2 >= len(agent2.path):
            return False

        current_time += PREDICTION_DT

    return False

def main():
    """Main simulation loop for Demo 8"""
    
    # --- Environment Setup ---
    static_obstacles = create_boundaries(space, SCREEN_WIDTH, SCREEN_HEIGHT)
    
    # --- Agent Setup ---
    start_1 = (100, 100)
    goal_1 = (700, 500)
    start_2 = (700, 450)
    goal_2 = (100, 100)

    agent1 = Agent(space, start_1, goal_1, static_obstacles, (SCREEN_WIDTH, SCREEN_HEIGHT), "blue")
    agent2 = Agent(space, start_2, goal_2, static_obstacles, (SCREEN_WIDTH, SCREEN_HEIGHT), "red")
    agents = [agent1, agent2]

    # --- Initial Planning (Non-Communicative) ---
    print("--- Initial Planning Phase ---")
    agent1.plan_path_initial()
    agent2.plan_path_initial() 
    
    # --- Cooldown timer ---
    replan_cooldown = 0.0

    # --- Simulation Loop ---
    running = True
    while running:
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        # Pymunk Physics Step
        dt = 1.0 / 60.0
        space.step(dt) # <-- Moved step to the top to get accurate dt

        # --- Re-planning Logic with Cooldown ---
        
        # 1. Tick down the cooldown timer
        if replan_cooldown > 0:
            replan_cooldown -= dt 

        # 2. Get current positions
        pos1 = agent1.body.position
        pos2 = agent2.body.position
        distance = pos1.get_distance(pos2)

        # 3. Check all trigger conditions
        is_in_warning_zone = distance < COMMUNICATION_TRIGGER_DISTANCE
        is_cooldown_over = replan_cooldown <= 0
        
        if is_in_warning_zone and is_cooldown_over:
            # 4. Only run expensive prediction if other checks pass
            if predict_collision(agent1, agent2, PREDICTION_HORIZON):
                
                print("\n--- HYBRID TRIGGER! ---")
                print(f"Agents {distance:.1f}px apart and on collision course. Re-planning...")
                
                # 5. Trigger the re-plan
                current_pos_2 = agent2.body.position
                new_planner = RRTPlanner(current_pos_2, agent2.goal, 
                                         agent2.static_obstacles, agent2.world_bounds)
                
                # --- THIS IS THE FIX ---
                # Check if agent1 has a valid path and index
                if agent1.path and agent1.path_index < len(agent1.path):
                    # Only give the planner the *remaining* path to avoid
                    remaining_path_for_agent1 = agent1.path[agent1.path_index:]
                    new_planner.set_dynamic_obstacles([remaining_path_for_agent1])
                
                agent2.planner = new_planner
                agent2.plan_path_non_destructive()
                
                # 6. Reset the cooldown timer!
                replan_cooldown = REPLAN_COOLDOWN_TIME

        # Update Agents (This just follows the current path)
        for agent in agents:
            agent.update()

        # Drawing
        screen.fill((255, 255, 255))  
        space.debug_draw(draw_options) 
        
        for agent in agents:
            pygame.draw.circle(screen, (0, 255, 0), (int(agent.goal.x), int(agent.goal.y)), 5)
            if agent.path:
                path_points = [(int(p.x), int(p.y)) for p in agent.path]
                pygame.draw.lines(screen, agent.shape.color, False, path_points, 2)

        pygame.display.flip()
        clock.tick(60)
        
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()